In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

np.random.seed(42)

# Wizyty

## Opis

- Data - klinika działa powiedzmy od roku czyli od 05.10.2019 
    - działają w weekendy
- godzina 
    - poniedziałęk - środa od 8 do 16
    - czwartek - piątek od 8 do 18
    - w weekendy od 9 do 13
- rodzaj wizyty - ???

Metoda:
- ile chcemy wizyt na dzień?
    - powiedzmy, że lekarz + personel średnio 5 dni w tygodniu pracuję
    - chirurg z racji trudnej pracy niech będzie w tej klinice 4 dni od poniedziałku do czwartku
    - lekarz 1 jest od poniedziałku do czwartku i w sobote
    - lekarz 2 od wtorku do piątku i niedzielę
    - Załóżmy, że mamy 4 rodzaje wizyt
         - 15 min z p-stwem 0.4
         - 30 min z p-stwem 0.3
         - 1h z p-stwem 0.2
         - 2h z p-stwem 0.1
    - Od 05.10.2019 do 09.05.2020 klinka się rozkręcała i prawdopodobieństwo, że klient przyjdzie na wizytę (wcześniej ją zarezerwował) wynosi 0.3
    - W pozostałych dniach to p-stwo wzrosło do 0.6
- Napisać generator liczb wizyt włącznie z rodzajem wizyt na dzień. Uwzględnić czy wizyta się odbyła czy nie
- !!! Przerwy pracowników są pomijalne i odbywają się ewentualnie jak nie ma żadnego pacjenta

- w ciągu tygodnia są 3 dni gdy jest 3 lekarzy, 2 dni jak jest 2 lekarzy i 2 dni jak jest jeden lekarz
- rozkłady wizyt do lekarzy są takie same

In [ ]:
def generuj_liczbe_wizyt_na_dzien(start, end, ilosc_godzin_na_dzien=[8, 8, 8, 10, 10, 4, 4], 
                                  p_wizyty=[0.4, 0.3, 0.2, 0.1],
                                  czas_wizyty=[0.25, 0.5, 1, 2],
                                  n_emps=[2, 3, 3, 3, 1, 1, 1]):
    
    '''start: datetime, początkowa data
    end: datetime, końcowa data
    ilosc_godzin_na_dzien: lista, ile w ciągu tego dnia pracują lekarze
    p_wizyty: lista, prawdopodobieństwo zajścia danego rodzaju wizyty
    czas_wizyty: float, ile (w godzinach) ma trwać dany rodzaj wizyty, lista musi być uporządkowana
    n_emps: lista; długości 7, , ilu pracowników jest w dany dzień w pracy'''
    
    t1 = pd.to_datetime(start, format="%d/%m/%Y")
    t2 = pd.to_datetime(end, format="%d/%m/%Y")
    df = pd.DataFrame({'Day':[], 'Wizyty':[], 'N_lekarzy':[]})
    
    # iterujemy po każdym dniu od t1 do t2
    for i in pd.date_range(t1, t2):
        # numer dnia tygodnia
        day_of_week = i.day_of_week
        
        # będziemy tu dodawać rozkład wizyt dla j-tego lekarza
        lista_n_wizyt = []
        # iterujemy po lekarzach obecnych w danym dniu
        for j in range(n_emps[day_of_week]):
            # pojedynczy rozkład wizyt u lekarza
            rodzaje_wizyt = []
            # czas spędzony w danym dniu w pracy
            czas = 0
            # dopóki jest w pracy to wykonuj
            while czas < ilosc_godzin_na_dzien[day_of_week]:
                
                # losowo wybieramy rodzaj wizyty w zależności od rozkładu prawdopodobieństw
                wizyta = np.random.choice(czas_wizyty, p=p_wizyty) 
                rodzaje_wizyt.append(wizyta)
                # dodajemy czas spędzony w robocie
                czas += wizyta
            lista_n_wizyt.append(rodzaje_wizyt)
        # dodajemy wyniki na dany dzień
        df = df.append(pd.DataFrame({'Day':[i], 'Wizyty':[lista_n_wizyt], 'N_lekarzy':[n_emps[day_of_week]]}))
    df = df.reset_index(drop=True)
    return df

In [ ]:
def utworz_date_godzine(df, dzisiaj_jest_lekarza=[[1, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [2], [1], [2]],
                        godziny_otwarcia=[8, 8, 8, 8, 8, 9, 9]):
    '''
    df: dataframe, wynik funkcji generuj_liczbe_wizyt_na_dzien
    dzisiaj_jest_lekarza: lista, zawiera id lekarza, który pracuje w danym dniu
    godziny_otwarcia: lista; długości 7, od której w dany dzień jest otwarta klinika'''
    
    result = pd.DataFrame({'Data':[], 'Godzina':[], 'Id lekarza':[]})
    
    # iterujemy po wierszach ramki
    for idx, row in df.iterrows():
        # iterujemy po każdym lekarzu z danego dnia
        for i in range(row[2]):
            # zaczynamy od danej godziny otwarcia
            t = pd.to_datetime(godziny_otwarcia[row[0].day_of_week], unit='h')
            # iterujemy po czasie trwania danej wizyty
            for time in row[1][i]:
                # dodajemy obecną date, godzinę i id lekarza który przyjmował pacjenta
                result = result.append(pd.DataFrame({'Data':[row[0]], 'Godzina':[str(t.time())], 'Czas trwania wizyty':[str(pd.to_timedelta(time, unit='h')).split()[2][:5]], 
                                                     'Id lekarza':[dzisiaj_jest_lekarza[row[0].day_of_week][i]]}))
                # następna wizyta
                t = t + pd.to_timedelta(time, unit='h')
    return result

## Symulacja pierwszego roku

In [ ]:
t1 = pd.to_datetime('05/10/2019', format="%d/%m/%Y")
t2 = pd.to_datetime('09/05/2020', format="%d/%m/%Y")

ilosci = generuj_liczbe_wizyt_na_dzien(t1, t2)
ilosci['N_lekarzy'] = ilosci['N_lekarzy'].astype(int)

ilosci

,Day,Wizyty,N_lekarzy
0,2019-10-05,"[[0.25, 2.0, 1.0, 0.5, 0.25]]",1
1,2019-10-06,"[[0.25, 0.25, 1.0, 0.5, 1.0, 0.25, 2.0]]",1
2,2019-10-07,"[[1.0, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.25,...",2
3,2019-10-08,"[[2.0, 0.25, 0.5, 0.25, 0.5, 0.5, 0.25, 2.0, 1...",3
4,2019-10-09,"[[0.25, 0.25, 1.0, 0.5, 0.25, 0.25, 0.25, 0.25...",3
...,...,...,...
213,2020-05-05,"[[0.25, 0.25, 1.0, 0.5, 0.5, 0.25, 0.5, 1.0, 1...",3
214,2020-05-06,"[[0.5, 0.5, 0.25, 0.5, 2.0, 2.0, 0.5, 0.25, 0....",3
215,2020-05-07,"[[1.0, 0.25, 0.5, 1.0, 1.0, 0.25, 0.5, 0.25, 0...",3
216,2020-05-08,"[[1.0, 1.0, 1.0, 0.25, 1.0, 1.0, 0.25, 0.5, 0....",1


In [ ]:
pierwszy_rok = utworz_date_godzine(ilosci)

In [ ]:
# prawdopodobieństwo czy ktoś zarezerwował wizytę
p_wizyty = 0.3
# ile wizyt było w pierwszym roku
n_wizyt = len(pierwszy_rok)
# tworzymy wektor 0 i 1 w zależności od prawdopodobieństwa wystąpienia wizyty
flaga_binarna = (np.random.rand(n_wizyt) <= p_wizyty).astype(int)
# tworzymy kolumne czy była wizyta
pierwszy_rok['Czy była wizyta?'] = flaga_binarna

In [ ]:
pierwszy_rok

,Data,Godzina,Id lekarza,Czas trwania wizyty,Czy była wizyta?
0,2019-10-05,09:00:00,1.0,00:15,0
0,2019-10-05,09:15:00,1.0,02:00,0
0,2019-10-05,11:15:00,1.0,01:00,1
0,2019-10-05,12:15:00,1.0,00:30,0
0,2019-10-05,12:45:00,1.0,00:15,1
...,...,...,...,...,...
0,2020-05-09,09:30:00,1.0,00:30,0
0,2020-05-09,10:00:00,1.0,02:00,0
0,2020-05-09,12:00:00,1.0,00:15,0
0,2020-05-09,12:15:00,1.0,00:15,0


## Symulowanie drugiego roku

In [ ]:
t1 = pd.to_datetime('10/05/2020', format="%d/%m/%Y")
t2 = pd.to_datetime('18/06/2021', format="%d/%m/%Y")

ilosci = generuj_liczbe_wizyt_na_dzien(t1, t2)
ilosci['N_lekarzy'] = ilosci['N_lekarzy'].astype(int)

drugi_rok = utworz_date_godzine(ilosci)

In [ ]:
# prawdopodobieństwo czy ktoś zarezerwował wizytę
p_wizyty = 0.6
# ile wizyt było w pierwszym roku
n_wizyt = len(drugi_rok)
# tworzymy wektor 0 i 1 w zależności od prawdopodobieństwa wystąpienia wizyty
flaga_binarna = (np.random.rand(n_wizyt) <= p_wizyty).astype(int)
# tworzymy kolumne czy była wizyta
drugi_rok['Czy była wizyta?'] = flaga_binarna

In [ ]:
drugi_rok

,Data,Godzina,Id lekarza,Czas trwania wizyty,Czy była wizyta?
0,2020-05-10,09:00:00,2.0,01:00,1
0,2020-05-10,10:00:00,2.0,00:30,1
0,2020-05-10,10:30:00,2.0,02:00,1
0,2020-05-10,12:30:00,2.0,00:30,1
0,2020-05-11,08:00:00,1.0,01:00,0
...,...,...,...,...,...
0,2021-06-18,14:15:00,2.0,01:00,1
0,2021-06-18,15:15:00,2.0,00:30,1
0,2021-06-18,15:45:00,2.0,00:15,0
0,2021-06-18,16:00:00,2.0,00:30,0


## Ostateczne

In [ ]:
# łączymy wizyty z peirwszego i drugiego roku, uwzględniając tylko te które się odbyły
wizyty = pierwszy_rok[pierwszy_rok['Czy była wizyta?'] == 1].append(drugi_rok[drugi_rok['Czy była wizyta?'] == 1])

wizyty = wizyty.iloc[:, :-1]
wizyty['Id lekarza'] = wizyty['Id lekarza'].astype(int) 

# Łączenie z rodzajem wizyty

In [ ]:
rodzaje_wizyt = pd.read_excel('dane do tabeli/rodzaj wizyty.xlsx')
rodzaje_wizyt.czas = rodzaje_wizyt.czas.apply(lambda x: str(x)[:-3])

In [ ]:
n_operacji_chirurga_15_min = len(wizyty[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '00:15')])
n_operacji_chirurga_30_min = len(wizyty[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '00:30')])
n_operacji_chirurga_1_h = len(wizyty[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '01:00')])
n_operacji_chirurga_2_h = len(wizyty[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '02:00')])

n_operacji_innych_15_min = len(wizyty[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '00:15')])
n_operacji_innych_30_min = len(wizyty[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '00:30')])
n_operacji_innych_1_h = len(wizyty[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '01:00')])
n_operacji_innych_2_h = len(wizyty[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '02:00')])

In [ ]:
wizyty['Rodzaj wizyty'] = 0

wizyty.loc[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '00:15'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 1) & (rodzaje_wizyt.czas == '00:15'), 'rodzaj wizyty'].sample(n_operacji_chirurga_15_min, replace=True).values
wizyty.loc[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '00:30'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 1) & (rodzaje_wizyt.czas == '00:30'), 'rodzaj wizyty'].sample(n_operacji_chirurga_30_min, replace=True).values
wizyty.loc[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '01:00'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 1) & (rodzaje_wizyt.czas == '01:00'), 'rodzaj wizyty'].sample(n_operacji_chirurga_1_h, replace=True).values
wizyty.loc[(wizyty['Id lekarza'] == 3) & (wizyty['Czas trwania wizyty'] == '02:00'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 1) & (rodzaje_wizyt.czas == '02:00'), 'rodzaj wizyty'].sample(n_operacji_chirurga_2_h, replace=True).values

wizyty.loc[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '00:15'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 0) & (rodzaje_wizyt.czas == '00:15'), 'rodzaj wizyty'].sample(n_operacji_innych_15_min, replace=True).values
wizyty.loc[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '00:30'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 0) & (rodzaje_wizyt.czas == '00:30'), 'rodzaj wizyty'].sample(n_operacji_innych_30_min, replace=True).values
wizyty.loc[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '01:00'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 0) & (rodzaje_wizyt.czas == '01:00'), 'rodzaj wizyty'].sample(n_operacji_innych_1_h, replace=True).values
wizyty.loc[(wizyty['Id lekarza'] != 3) & (wizyty['Czas trwania wizyty'] == '02:00'), 'Rodzaj wizyty'] = rodzaje_wizyt.loc[(rodzaje_wizyt['czy chirurg'] == 0) & (rodzaje_wizyt.czas == '02:00'), 'rodzaj wizyty'].sample(n_operacji_innych_2_h, replace=True).values

# Łączenie z id zwierzaka

#### Zwierzę może zostać zarejestrowane nie będąc jeszcze na wizycie

In [ ]:
zwierzeta = pd.read_csv('dane do tabeli/zwierzeta.csv', index_col='Unnamed: 0')

In [ ]:
wizyty

,Data,Godzina,Id lekarza,Czas trwania wizyty,Rodzaj wizyty
0,2019-10-05,11:15:00,1,01:00,testy alergiczne
0,2019-10-05,12:45:00,1,00:15,kontrolna
0,2019-10-06,09:00:00,2,00:15,badanie krwi
0,2019-10-07,09:00:00,1,00:15,zabieg higieniczny drobny
0,2019-10-07,09:15:00,1,00:15,chipowanie
...,...,...,...,...,...
0,2021-06-18,10:00:00,2,01:00,antykoncepcja
0,2021-06-18,11:00:00,2,00:30,EKG
0,2021-06-18,12:00:00,2,00:15,zabieg higieniczny drobny
0,2021-06-18,14:15:00,2,01:00,zabieg ortopedyczny


In [ ]:
ilosc_wizyt_na_dzien = wizyty.groupby('Data')['Data'].count()

In [ ]:
zarejestrowane_zwierzeta = zwierzeta.groupby('Data rejestracji')['id_zwierzęcia'].agg(lambda x: list(x)).cumsum()

zarejestrowane_zwierzeta.index = pd.to_datetime(zarejestrowane_zwierzeta.index)

zarejestrowane_zwierzeta = zarejestrowane_zwierzeta.reindex(ilosc_wizyt_na_dzien.index, method='bfill')

number_of_nans = len(zarejestrowane_zwierzeta[zarejestrowane_zwierzeta.isna()])

id_zwierzecia = zarejestrowane_zwierzeta[-number_of_nans -1]

for i in range(number_of_nans + 1):
    zarejestrowane_zwierzeta[-i] = id_zwierzecia

In [ ]:
for data, n_wizyt, id_list in zip(zarejestrowane_zwierzeta.index, ilosc_wizyt_na_dzien, zarejestrowane_zwierzeta):
    wizyty.loc[wizyty['Data'] == data, 'id_zwierzęcia'] = np.random.choice(id_list, n_wizyt)

In [ ]:
wizyty['id_zwierzęcia'] = wizyty['id_zwierzęcia'].astype(int)

In [ ]:
wizyty.columns

Index(['Data', 'Godzina', 'Id lekarza', 'Czas trwania wizyty', 'Rodzaj wizyty',
       'id_zwierzęcia'],
      dtype='object')

In [ ]:
wizyty = wizyty.reset_index(drop=True).reset_index().rename(columns={'index':'id_wizyty'})

# Zapisanie

In [ ]:
eutanazje = wizyty[wizyty['Rodzaj wizyty']=='eutanazja'].groupby('id_zwierzęcia')['Data'].idxmin().to_frame()

In [ ]:
wizyty[(wizyty['id_zwierzęcia']==0 )& (wizyty['id_wizyty']>3110) ]

,id_wizyty,Data,Godzina,Id lekarza,Czas trwania wizyty,Rodzaj wizyty,id_zwierzęcia
3419,3419,2020-08-31,11:30:00,3,00:30,zabieg chirurgiczny drobny,0
3492,3492,2020-09-03,08:00:00,3,02:00,operacja w znieczuleniu częściowym,0
3554,3554,2020-09-08,15:30:00,3,00:30,zabieg chirurgiczny drobny,0
4162,4162,2020-10-15,14:30:00,3,00:15,konsultacja chirurgiczna,0
5440,5440,2021-01-07,16:45:00,2,00:15,szczepienie,0
5953,5953,2021-02-09,10:45:00,1,00:30,kroplówka witaminowa,0
6391,6391,2021-03-09,13:45:00,1,01:00,testy alergiczne,0
6807,6807,2021-04-06,11:15:00,2,00:30,RTG,0
7367,7367,2021-05-13,11:45:00,1,00:30,kroplówka witaminowa,0


In [ ]:
wizyty_do_wywalenia = []
for idx, id in eutanazje.iterrows():
    id = int(id)
    do_wywalenia = wizyty[(wizyty['id_zwierzęcia']==idx)& (wizyty['id_wizyty']>id)].index
    wizyty_do_wywalenia = wizyty_do_wywalenia + list(do_wywalenia)
    

In [ ]:
wizyty_do_wywalenia

[4519,
 6048,
 6110,
 6350,
 7258,
 7496,
 3346,
 3500,
 4240,
 5041,
 5756,
 6819,
 6929,
 4902,
 5466,
 5510,
 5590,
 5721,
 6232,
 6396,
 7631,
 7883,
 3878,
 4521,
 5004,
 6673,
 6993,
 1509,
 1566,
 1682,
 3508,
 4625,
 6040,
 7829,
 3385,
 3390,
 3662,
 4002,
 6200,
 3847,
 4536,
 5526,
 6872,
 7598,
 2191,
 2800,
 3930,
 5680,
 6308,
 6738,
 7050,
 7855,
 4824,
 6314,
 735,
 787,
 1039,
 2508,
 3169,
 7769,
 2272,
 3119,
 3322,
 3961,
 4100,
 7083,
 2550,
 3269,
 4493,
 5334,
 5734,
 6067,
 7568,
 6535,
 6924,
 1538,
 1563,
 1578,
 1672,
 1907,
 2302,
 2307,
 2327,
 2803,
 3389,
 4369,
 5977,
 6083,
 6233,
 6254,
 6401,
 6718,
 7209,
 7386,
 7672,
 2498,
 2915,
 3175,
 3945,
 5270,
 6024,
 3984,
 4198,
 4698,
 4776,
 4983,
 5104,
 7108,
 2106,
 2665,
 2787,
 3225,
 3570,
 3834,
 4574,
 4958,
 5524,
 7009,
 7592,
 4385,
 5588,
 6719,
 7174,
 4021,
 4479,
 4744,
 7085,
 7183,
 7530,
 7739,
 4056,
 4109,
 6457,
 6513,
 3599,
 3748,
 6941,
 6928,
 555,
 762,
 763,
 784,
 797,
 842,


In [ ]:
len(wizyty_do_wywalenia)

1241

In [ ]:
wizyty = wizyty.drop(wizyty_do_wywalenia)

In [ ]:
wizyty = wizyty.reset_index(drop=True).reset_index().rename(columns={'index':'id_wizyty'})

In [ ]:
wizyty.to_csv('wizyty.csv')

# Do transakcji

In [ ]:
wizyty_u_babci = wizyty.copy()

In [ ]:
wizyty_u_babci = pd.merge(wizyty_u_babci, rodzaje_wizyt, how='left', left_on='Rodzaj wizyty', right_on='rodzaj wizyty')

In [ ]:
wizyty_u_babci

,id_wizyty,id_wizyty,Data,Godzina,Id lekarza,Czas trwania wizyty,Rodzaj wizyty,id_zwierzęcia,czas,rodzaj wizyty,koszt,czy chirurg
0,0,0,2019-10-05,11:15:00,1,01:00,testy alergiczne,772,01:00,testy alergiczne,150,0
1,1,1,2019-10-05,12:45:00,1,00:15,kontrolna,486,00:15,kontrolna,40,0
2,2,2,2019-10-06,09:00:00,2,00:15,badanie krwi,1109,00:15,badanie krwi,60,0
3,3,3,2019-10-07,09:00:00,1,00:15,zabieg higieniczny drobny,894,00:15,zabieg higieniczny drobny,50,0
4,4,4,2019-10-07,09:15:00,1,00:15,chipowanie,594,00:15,chipowanie,60,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6685,6685,7925,2021-06-18,08:00:00,2,02:00,eutanazja,552,02:00,eutanazja,200,0
6686,6686,7926,2021-06-18,10:00:00,2,01:00,antykoncepcja,933,01:00,antykoncepcja,150,0
6687,6687,7927,2021-06-18,11:00:00,2,00:30,EKG,755,00:30,EKG,80,0
6688,6688,7928,2021-06-18,12:00:00,2,00:15,zabieg higieniczny drobny,811,00:15,zabieg higieniczny drobny,50,0


In [ ]:
wizyty_u_babci[list(wizyty.columns) + ['koszt']].to_csv('wizyty_inner_join_kwota.csv')